In [11]:
print('hello')

hello


In [12]:
import pandas as pd
import csv
import numpy as np
import math
import random
from collections import Counter

In [13]:
banking_df = pd.read_csv("/kaggle/input/bank-full/bank-full.csv", delimiter=";")
banking_df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown,yes
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown,yes
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,yes
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown,no


In [14]:
# in 1 dong du lieu trong data frame banking
print(banking_df.head())
print('-----------------------------------')

#co bao nhieu cot la du lieu dang phan loai
print(banking_df.dtypes.value_counts())
print('-----------------------------------')

# co bao nhieu cot o features va cot predict
print(banking_df.shape)
print('-----------------------------------')

# tinh toan xem co bao nhieu du lieu bi miss o moi cot
print(banking_df.isna().sum())
print('-----------------------------------')

# tinh va in ra so khach hang co tiet kiem ky han va ko gui tien
print(banking_df['y'].value_counts())
print('-----------------------------------')

# min max summary dataset
print(banking_df.describe())
print('-----------------------------------')

# co bao nhieu khach hang duoi 30
under_30 = banking_df[banking_df['age']<30]
Cus_under_30 = under_30['age'].count()
print(f'so luong khach hang duoi 30 tuoi: {Cus_under_30}')

   age           job  marital  education default  balance housing loan  \
0   58    management  married   tertiary      no     2143     yes   no   
1   44    technician   single  secondary      no       29     yes   no   
2   33  entrepreneur  married  secondary      no        2     yes  yes   
3   47   blue-collar  married    unknown      no     1506     yes   no   
4   33       unknown   single    unknown      no        1      no   no   

   contact  day month  duration  campaign  pdays  previous poutcome   y  
0  unknown    5   may       261         1     -1         0  unknown  no  
1  unknown    5   may       151         1     -1         0  unknown  no  
2  unknown    5   may        76         1     -1         0  unknown  no  
3  unknown    5   may        92         1     -1         0  unknown  no  
4  unknown    5   may       198         1     -1         0  unknown  no  
-----------------------------------
object    10
int64      7
dtype: int64
-----------------------------------


In [15]:
# chuyen doi du lieu cot y no-> 0 va yes -> 1

# banking_df['y'] =  banking_df['y'].apply(lambda x: 1 if "yes" in x else 0)
banking_df['y'] = banking_df['y'].replace({'no': 0, 'yes': 1})
print(banking_df['y'])
print(banking_df['y'].value_counts())

0        0
1        0
2        0
3        0
4        0
        ..
45206    1
45207    1
45208    1
45209    0
45210    0
Name: y, Length: 45211, dtype: int64
0    39922
1     5289
Name: y, dtype: int64


In [22]:
# chia tap data thanh 2 train va test

train_df = banking_df.sample(frac=0.85, random_state=2023)
test_df = banking_df.drop(train_df.index)

# tach du lieu ko co nhan o tap va tap test
print(train_df['y'].value_counts(normalize=True))
print(test_df['y'].value_counts(normalize=True))

X_train = train_df.drop('y', axis=1)
y_train = train_df['y']

X_test = test_df.drop('y', axis=1)
y_test = test_df['y']

0    0.882849
1    0.117151
Name: y, dtype: float64
0    0.883958
1    0.116042
Name: y, dtype: float64


In [21]:
# viet ham tinh khoang cach euclid
#X_train["distance"] = abs(X_train['age'] - X_test.iloc[2023]['age'])
print(X_train["distance"])
def knn(feature, single_test_input, k):
  X_train["distance"] = abs(X_train[feature] - single_test_input[feature])
  prediction = y_train[X_train["distance"].nsmallest(n=k).index].mode()[0]
  return prediction

model_prediction = knn("age", X_test.iloc[2023], 3)
print("Predict label:",model_prediction)
print("Actual label",y_test.iloc[2023])

36454     2
27406     2
26095     1
5573      5
39103     1
         ..
32846    11
42222    14
28857     1
12241    10
24883    10
Name: distance, Length: 38429, dtype: int64
Predict label: 0
Actual label 0


In [23]:
X_test["age_predicted_y"] = X_test.apply(lambda x:knn("age", x, 3), axis=1)

print(X_test["age_predicted_y"])

1        0
3        0
14       0
15       0
26       0
        ..
45159    0
45163    0
45178    0
45199    0
45210    0
Name: age_predicted_y, Length: 6782, dtype: int64


In [24]:
model_accuracy = (X_test["age_predicted_y"] == y_test).value_counts(normalize=True)[True]*100

print(model_accuracy)

84.8864641698614


In [25]:
#lam tiep cho feature campaign
X_test["campaign_predicted_y"] = X_test.apply(lambda x:knn("campaign", x, 3), axis=1)

model_accuracy = (X_test["age_predicted_y"] == y_test).value_counts(normalize=True)[True]*100

print(model_accuracy)

84.8864641698614


In [26]:
def knn(features, single_test_input, k):
  squared_distance = 0
  for feature in features:
    squared_distance += (X_train[feature] - single_test_input[feature])**2
    X_train['distance'] = squared_distance**0.5
    prediction = y_train[X_train["distance"].nsmallest(n=k).index].mode()[0]
    return prediction

model_predection = knn(['age', 'campaign', 'marital_married', 'marital_single'], X_test.iloc[408], 3)

print(model_predection)
print(y_test.iloc[408])

0
0


In [27]:
X_test["predicted_y"] = X_test.apply(lambda x: knn(['age', 'campaign', 'marital_married', 'marital_single'], x, 3), axis=1)

model_accuracy = (X_test["predicted_y"] == y_test).value_counts(normalize=True)[True]*100

print(model_accuracy)

84.8864641698614
